In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('ALL-INDICES.csv')
df

In [ ]:
df = df[df['year']== 2022]
df

In [ ]:
df = df[df['source_type']== 'reviews']

In [ ]:
df = df[df['source_site']== 'banki.ru']

In [ ]:
df = df[['year', 'quarter', 'bank_id', 'INDEX_SAFE', 'TOTAL', 'NEUTRAL', 'POS','NEG']]

In [ ]:
mean_index_safe_per_year = df.groupby(['year', 'bank_id'])['INDEX_SAFE'].mean().reset_index()

# Merge the mean INDEX_SAFE back to the original DataFrame
df = pd.merge(df, mean_index_safe_per_year, on=['year', 'bank_id'], suffixes=('', '_mean'))

# Rename the new column for clarity
df.rename(columns={'INDEX_SAFE_mean': 'mean_index_safe'}, inplace=True)
df = df[['year', 'bank_id', 'mean_index_safe']]
# Display the DataFrame
df.head()

In [ ]:
df = df.drop_duplicates()


In [ ]:
bank_names = pd.read_csv('banki_ru_NAMES.csv')

In [ ]:
bank_names

In [ ]:
banks_2022 = pd.read_csv('top2022.csv') #народный рейтинг banki.ru, 2022

In [ ]:
banks_2022

In [ ]:
banki_ru = pd.merge(banks_2022, bank_names, left_on='Bank', right_on='bank_name')

banki_ru

In [ ]:
df

In [ ]:
index_sorted = df.sort_values(by='mean_index_safe', ascending=False)
index_sorted

In [ ]:
df_sorted_banks = pd.merge(index_sorted, bank_names, on='bank_id' )

In [ ]:
df_sorted_banks

In [ ]:
df_sorted_banks['Rank'] = df_sorted_banks['mean_index_safe'].rank(method='first', ascending=False).astype(int)
df_sorted_banks

In [ ]:
banki_ru

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
df_compare = pd.merge(banki_ru[['bank_id', 'Rank', 'Bank', 'Rating']], df_sorted_banks[['bank_id', 'Rank', 'mean_index_safe']], on='bank_id')
df_compare

In [ ]:
# Calculate Spearman’s Rank Correlation Coefficient (ρ)
rho, _ = spearmanr(df_compare['Rank_x'], df_compare['Rank_y'])

In [ ]:
rho

In [ ]:
df_compare['Rating'] = df_compare['Rating'].str.replace(',', '.').astype(float)
df_compare['Rating_norm'] = df_compare['Rating']/100

In [ ]:
df_compare['index_norm'] = (df_compare['mean_index_safe'] + 1) / 2

In [ ]:

df_compare['index_norm'].max()

In [ ]:
df_compare['Rating_norm'].max()

In [ ]:
df_compare.head(200)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
normalized_0_100 = df_compare['Rating_norm']
normalized_neg1_1 = df_compare['index_norm']

# Pearson Correlation Coefficient
pearson_corr = np.corrcoef(normalized_0_100, normalized_neg1_1)[0, 1]

# Mean Absolute Error (MAE)
mae = mean_absolute_error(normalized_0_100, normalized_neg1_1)

# Mean Squared Error (MSE)
mse = mean_squared_error(normalized_0_100, normalized_neg1_1)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Create a DataFrame to display the measures
measures = {
    'Measure': ['Pearson Correlation Coefficient', 'Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)'],
    'Value': [pearson_corr, mae, mse, rmse]
}

measures_df = pd.DataFrame(measures)

In [ ]:
measures_df